In [98]:
pwd

'/scratch/ar6381/CV/project/data/sample'

In [12]:
from itertools import chain
import numpy as np
import pandas as pd
import os
from glob import glob

In [13]:
cd ..

/scratch/ar6381/CV/project/data


In [90]:
metadata = pd.read_csv(os.path.join('sample_labels.csv'))

In [91]:
metadata.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y
0,00000013_005.png,Emphysema|Infiltration|Pleural_Thickening|Pneu...,5,13,060Y,M,AP,3056,2544,0.139,0.139
1,00000013_026.png,Cardiomegaly|Emphysema,26,13,057Y,M,AP,2500,2048,0.168,0.168
2,00000017_001.png,No Finding,1,17,077Y,M,AP,2500,2048,0.168,0.168
3,00000030_001.png,Atelectasis,1,30,079Y,M,PA,2992,2991,0.143,0.143
4,00000032_001.png,Cardiomegaly|Edema|Effusion,1,32,055Y,F,AP,2500,2048,0.168,0.168


In [92]:
file_system_scan = {os.path.basename(x): x for x in glob(os.path.join('images', '*.png'))}
if len(file_system_scan) != metadata.shape[0]:
    raise Exception(
        'ERROR: Different number metadata records and png files.'.format())

metadata['path'] = metadata['Image Index'].map(file_system_scan.get)
print('Total x-ray records:{}.'.format((metadata.shape[0])))

Total x-ray records:5606.


In [93]:
metadata['Finding Labels'] = metadata['Finding Labels'].map(
        lambda x: x.replace('No Finding', 'No_Finding'))

In [94]:
labels = np.unique(list(chain(*metadata['Finding Labels'].map(lambda x: x.split('|')).tolist())))
labels = [x for x in labels if len(x) > 0]

In [95]:
print(labels)

['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'No_Finding', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']


In [96]:
for c_label in labels:
    if len(c_label) > 1:  # leave out empty labels
        metadata[c_label] = metadata['Finding Labels'].map(
            lambda finding: 1.0 if c_label in finding else 0)
labels = [c_label for c_label in labels if metadata[c_label].sum() > 1000]

In [97]:
metadata.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,...,Emphysema,Fibrosis,Hernia,Infiltration,Mass,No_Finding,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax
0,00000013_005.png,Emphysema|Infiltration|Pleural_Thickening|Pneu...,5,13,060Y,M,AP,3056,2544,0.139,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
1,00000013_026.png,Cardiomegaly|Emphysema,26,13,057Y,M,AP,2500,2048,0.168,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00000017_001.png,No_Finding,1,17,077Y,M,AP,2500,2048,0.168,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,00000030_001.png,Atelectasis,1,30,079Y,M,PA,2992,2991,0.143,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00000032_001.png,Cardiomegaly|Edema|Effusion,1,32,055Y,F,AP,2500,2048,0.168,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [80]:
sample_weights = metadata['Finding Labels'].map(
        lambda x: len(x.split('|')) if len(x) > 0 else 0).values + 4e-2
sample_weights /= sample_weights.sum()
metadata_sample = metadata.sample(80000, weights=sample_weights)

labels_count = [(c_label, int(metadata_sample[c_label].sum()))
                for c_label in labels]

print('Labels ({}:{})'.format((len(labels)), (labels_count)))
print('Total x-ray records:{}.'.format((metadata_sample.shape[0])))

Labels (14:[('Atelectasis', 9656), ('Cardiomegaly', 2246), ('Consolidation', 3973), ('Edema', 1984), ('Effusion', 11266), ('Emphysema', 2122), ('Fibrosis', 1383), ('Infiltration', 15959), ('Mass', 4820), ('No_Finding', 40288), ('Nodule', 5101), ('Pleural_Thickening', 2819), ('Pneumonia', 1268), ('Pneumothorax', 4325)])
Total x-ray records:80000.


In [81]:
metadata=metadata_sample

In [99]:
import shutil
for index, data in metadata.iterrows():
    labels = data['Finding Labels'].split('|')
    path = data['path']
    for label in labels:
        labelFolder = os.path.join('data', label)
        if not os.path.exists(labelFolder):
            os.makedirs(labelFolder)
        # construct the destination image path and copy the current
        # image to it
        destination = os.path.join(labelFolder, data['Image Index'])
        shutil.copy(path, destination)
    #os.remove(path)

In [100]:
metadata.head(10)

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,...,Emphysema,Fibrosis,Hernia,Infiltration,Mass,No_Finding,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax
0,00000013_005.png,Emphysema|Infiltration|Pleural_Thickening|Pneu...,5,13,060Y,M,AP,3056,2544,0.139,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
1,00000013_026.png,Cardiomegaly|Emphysema,26,13,057Y,M,AP,2500,2048,0.168,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00000017_001.png,No_Finding,1,17,077Y,M,AP,2500,2048,0.168,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,00000030_001.png,Atelectasis,1,30,079Y,M,PA,2992,2991,0.143,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00000032_001.png,Cardiomegaly|Edema|Effusion,1,32,055Y,F,AP,2500,2048,0.168,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,00000040_003.png,Consolidation|Mass,3,40,068Y,M,PA,2500,2048,0.168,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,00000042_002.png,No_Finding,2,42,071Y,M,AP,3056,2544,0.139,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,00000057_001.png,No_Finding,1,57,071Y,M,AP,3056,2544,0.139,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,00000061_002.png,Effusion,2,61,077Y,M,PA,2992,2991,0.143,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,00000061_019.png,No_Finding,19,61,077Y,M,AP,3056,2544,0.139,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
